In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

%matplotlib inline
# ---- settings ----
import json
Settings = json.load(open('../settings.txt'))
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
sys.path.insert(0, '../'); sys.path.insert(0, '../samples')
from paf_loader import Loader
import numpy as np
import cv2
from cselect import color as cs
from time import time
import mvpose.data.kth_football2 as kth
from mvpose import pose
from mvpose.evaluation import pcp
from mvpose import settings
from os.path import isfile, join

data_root = Settings['data_root']

params = settings.get_settings(scale_to_mm=1000,
                               min_nbr_joints=5,
                               max_epi_distance=25)

import mvpose.data.skeleton_augmentation as ska
model_path = '../data/model_poseprediction.h5'
gen = ska.LimbGenerator(model_path, params.scale_to_mm)

# ----------------------------------
# ==================================
seq1_zipname = 'player2sequence1.zip'
seq1_dir = 'Sequence 1'
EXP_NBR = "exp01"
# ==================================
# ----------------------------------


# ====================================================
# Extractor Functions
# ====================================================
def extract_best(pos3d, Humans, alpha):
    """
        Extracts the best fitting human for the calculation
    """
    larms = 0
    uarms = 0
    ulegs = 0
    llegs = 0
    all_parts = 0
    
    for h in Humans:
        r = pcp.evaluate(pos3d, Humans[0], alpha)
        if all_parts < r.all_parts:
            larms = r.lower_arms
            uarms = r.upper_arms
            ulegs = r.upper_legs
            llegs = r.lower_legs
    
    return {
        "larms": larms,
        "uarms": uarms,
        "ulegs": ulegs,
        "llegs": llegs
    }


def apply(frame, alpha, loader):
    """
    """
    global seq1_zipname
    global seq1_zip
    global EXP_NBR
    Im, Calib, pos2d, pos3d = kth.get(
        data_root, seq1_zipname, seq1_dir, frame, player=2)
    heatmaps, pafs = loader.load_confidence_map_and_paf(
        'kth' + seq1_zipname[0:-4], Im, frame, dir=Settings['tmp'])
    
    
#     fname = join('/tmp', EXP_NBR + "_frame" + str(frame) + '.npy')
#     if isfile(fname):
#         detections = np.load(fname)
#     else:
    detections = pose.estimate(Calib, heatmaps, pafs, 
                           settings=params)
#         np.save(fname, detections)
    
    Humans = kth.transform3d_from_mscoco(detections)
    result1 = extract_best(pos3d, Humans, alpha)
    
    # ---------------------------------
    
    
#     detections = gen.apply(detections)
#     Humans = kth.transform3d_from_mscoco(detections)
#     result2 = extract_best(pos3d, Humans, alpha)
    
    del Im
    del Calib
    del pos2d
    del pos3d
    del heatmaps
    del pafs
    del detections
    del Humans
    
    #return result1, result2
    return result1

/home/tanke/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def run_experiment():
    
    loader = Loader()

    alpha = 0.5

    larms = []
    uarms = []
    llegs = []
    ulegs = []

    # 0 .. 214
    for frame in range(0, 214):
        _start = time()
        result = apply(frame, alpha, loader)
        larms.append(result['larms'])
        uarms.append(result['uarms'])
        llegs.append(result['llegs'])
        ulegs.append(result['ulegs'])

        _end = time()
        print('frame ' + str(frame) + ', elapsed', _end - _start)
    
    return larms, uarms, llegs, ulegs

larms, uarms, llegs, ulegs = run_experiment()

frame 0, elapsed 3.535351514816284
frame 1, elapsed 3.0612666606903076
frame 2, elapsed 2.9694557189941406
frame 3, elapsed 2.9357306957244873
frame 4, elapsed 3.1796975135803223
frame 5, elapsed 3.1109416484832764
frame 6, elapsed 3.037367105484009
frame 7, elapsed 3.1096608638763428
frame 8, elapsed 2.967550277709961
frame 9, elapsed 3.001603126525879
frame 10, elapsed 3.0978221893310547
frame 11, elapsed 2.9869911670684814
frame 12, elapsed 3.1336171627044678
frame 13, elapsed 3.1381900310516357
frame 14, elapsed 3.176112651824951
frame 15, elapsed 2.9934754371643066
frame 16, elapsed 3.295844554901123
frame 17, elapsed 2.632218360900879
frame 18, elapsed 1.5204217433929443
frame 19, elapsed 3.041163921356201
frame 20, elapsed 3.016808032989502
frame 21, elapsed 3.0224075317382812
frame 22, elapsed 2.111865520477295
frame 23, elapsed 3.0205061435699463
frame 24, elapsed 2.9355404376983643
frame 25, elapsed 2.972856283187866
frame 26, elapsed 3.12060809135437
frame 27, elapsed 0.8032

## No unary (simple)
* upper arms:	 0.955607476635514
* lower arms:	 0.822429906542056
* upper legs:	 0.9439252336448598
* lower legs:	 0.883177570093458

# With unary (simple)
* upper arms:	 0.955607476635514
* lower arms:	 0.8247663551401869
* upper legs:	 0.9439252336448598
* lower legs:	 0.897196261682243

# Like in Master thesis with Xi = 0
* upper arms:	 0.9532710280373832
* lower arms:	 0.7990654205607477
* upper legs:	 0.9602803738317757
* lower legs:	 0.9345794392523364

# Like in Master thesis with Xi = 0.5
* upper arms:	 0.955607476635514
* lower arms:	 0.8271028037383178
* upper legs:	 0.9439252336448598
* lower legs:	 0.9065420560747663

# Like in Master thesis with Xi = 0.7 (x)
* upper arms:	 0.9532710280373832
* lower arms:	 0.8341121495327103
* upper legs:	 0.9485981308411215
* lower legs:	 0.9135514018691588
* avg:		 0.9123831775700935

# Like in Master thesis without unary
* upper arms:	 0.9532710280373832
* lower arms:	 0.8341121495327103
* upper legs:	 0.9462616822429907
* lower legs:	 0.8995327102803738
* avg:		 0.9082943925233645

In [3]:
print("upper arms:\t", np.mean(uarms))
print("lower arms:\t", np.mean(larms))
print("upper legs:\t", np.mean(ulegs))
print("lower legs:\t", np.mean(llegs))
print('avg:\t\t', np.mean([
    np.mean(uarms), np.mean(larms), np.mean(ulegs), np.mean(llegs)
]))

upper arms:	 0.9532710280373832
lower arms:	 0.8341121495327103
upper legs:	 0.9485981308411215
lower legs:	 0.9135514018691588
avg:		 0.9123831775700935
